# Python и интернет. Модуль requests

**План**:

1. Запросы
2. Requests
4. bs4
5. Задание на семинар

## Как выкачать интернет
Современный Интернет предоставляет лингвистам большое количество языковых данных: электронные газеты и журналы, блоги, форумы, социальные сети и т.д. Например, можно найти в сети много-много текстов и собрать корпус, или найти все газетные статьи и блог-посты про какую-нибудь корпорацию и проанализировать тональность сообщений. Сейчас мы научимся заниматься выкачиванием страниц из интернета с помощью Python.

Для скачивания HTML-страниц в питоне есть несколько библиотек **urllib** и **requests**

## Requests

Можно почитать еще [тут](https://realpython.com/python-requests/)


Допустим, мы хотим скачать главную страницу Хабрахабра.

На самом деле, когда мы хотим открыть какую-то страницу в интернете, наш браузер отправляет на сервер **запрос** ("Привет, сервер! я хочу код страницы по вот такому адресу!"), а сервер затем отправляет ответ ("Привет! Вот код страницы: ...").
Чтобы получить страницу через питон, нужно сформировать **запрос** на сервер так же, как это делает браузер:

In [1]:
import requests

In [2]:
response = requests.get("https://habr.com/ru/")

В response теперь лежит отет сервера. Это не прост html-код страницы, а еще дополнительная информация. Если мы просто выведем этот response, нам покажется только код (200 - все ок).

In [3]:
response

<Response [200]>

In [4]:
response.status_code

200

А вот в атрибуте text уже лежит html-код

In [8]:
print(response.text[:700])

<!DOCTYPE html>
<html lang="ru" data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D">
<head >
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width,initial-scale=1.0,viewport-fit=cover,maximum-scale=1,user-scalable=0">
  <meta name="referrer" content="unsafe-url">
  <title>Все статьи подряд / Хабр</title>
  <style>
    /* cyrillic-ext */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format('woff2');
      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;
 


Иногда сайт блокирует запросы, если их посылает не настоящий браузер с пользователем, а какой-то бот (например, так делает Гугл или Википедия). Иногда сайты присылают разные версии страниц, разным браузерам.  
По этим причинам полезно бывает писать скрипт, который умеет притворяться то одним, то другим браузером.
Когда мы пытаемся получить страницу в питоне, наш код по умолчанию честно сообщает серверу, что он является программой на питоне. Он говорит что-то вроде "Привет, я Python-urllib/3.5".
Но можно, например, представиться Мозиллой:

In [9]:
url = 'https://habr.com/ru/'  # адрес страницы, которую мы хотим скачать
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером

response = requests.get("https://habr.com/ru/", headers={'User-Agent':user_agent})

Или использовать специальную библиотеку

In [10]:
!pip install fake_useragent

In [11]:
from fake_useragent import UserAgent

In [12]:
user_agent = UserAgent().chrome
user_agent

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

In [13]:
response = requests.get("https://habr.com/ru/", headers={'User-Agent':user_agent})

Можно посмотреть, что еще можно достать из response.

Функция ```dir``` выдает список методов и параметров объекта.

In [ ]:
[i for i in dir(response) if not i.startswith("_")]

['apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

Кодировка

In [ ]:
response.encoding

'utf-8'

Заголовки (техническая информация)

In [14]:
dict(response.headers)

{'Server': 'QRATOR',
 'Date': 'Thu, 02 Nov 2023 09:33:36 GMT',
 'Content-Type': 'text/html; charset=utf-8',
 'Transfer-Encoding': 'chunked',
 'Connection': 'keep-alive',
 'Keep-Alive': 'timeout=15',
 'Vary': 'Accept-Encoding, Accept-Encoding',
 'X-DNS-Prefetch-Control': 'off',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-Download-Options': 'noopen',
 'X-Content-Type-Options': 'nosniff',
 'X-XSS-Protection': '1; mode=block',
 'Cache-Control': 'no-store, no-cache, must-revalidate',
 'Pragma': 'no-cache',
 'Expires': '0',
 'ETag': 'W/"543c9-XyEacIKmX3mPK6ZOSbNYdDOZQKY"',
 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains',
 'X-Request-Id': '7a79a474a4681731e664e9d0acf4cb0c',
 'X-Request-Geoip-Country-Code': 'US',
 'X-Request-Detected-Device': 'desktop',
 'Content-Encoding': 'gzip'}

Адрес запроса

In [15]:
response.url

'https://habr.com/ru/articles/'

In [16]:
print(response.text[:300])

<!DOCTYPE html>
<html lang="ru" data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D">
<head >
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width,initial-scale=1.0,viewport-fit=cover,maximum-scale=1,user-scalable=0">
  <meta name="referrer" content="unsafe-url">
  <titl


Ура, всё на месте!

Но что всё это значит? Что такое html и как вообще из него доставать какую-то информацию?

Ответ: по **тегам**! Например, в куске html сверху есть теги `<title> </title>` (теги всегда обрамляют с двух сторон то, что находится под этим тегом). В `<title>` в данном случае лежит заголовок этой интернет-страницы.

Существует несколько вариантов, как достать что-то из определенного тега, например, достать заголовок:

1. регулярные выражения (плохой вариант)
2. специальные библиотеки питона, например, BeautifulSoup (bs4) или lxml (хороший вариант)

## BeautifulSoup

Код страницы парсится как иерархия тегов (как они есть в html коде, один вложен в другой) и можно искать элементы с помощью разных методов, использовать атрибуты и т.д.

In [17]:
from bs4 import BeautifulSoup

Сначала инициализируем объект BeautifulSoup. Потом применим метод find и в скобочках укажем теги, по которым ищем. У некоторых тегов в html (как и в нашем случае) бывает еще и class и какие-нибудь еще атрибуты. Такие вещи мы задаем словариком.

Этот запрос вернёт нам только первый заголовок. То есть первое вхождение такого тега в нашем html файле.

In [20]:
soup = BeautifulSoup(response.text, 'html.parser')  # инициализируем (создаем) soup

post = soup.find('h2', {'class': 'tm-title tm-title_h2'})
print(post.get_text(), end="\n\n\n")
print(post.prettify())

Обзор Llemma: новая математическая open-source модель


<h2 class="tm-title tm-title_h2">
 <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/companies/mts_ai/articles/771476/">
  <span>
   Обзор Llemma: новая математическая open-source модель
  </span>
 </a>
</h2>



Но мы хотим получить все заголовки постов! Метод find_all возвращает массив всех элементов с тегом указанным в скобках. По нему можно итерироваться.

In [21]:
for post in soup.find_all('h2', {'class': 'tm-title tm-title_h2'})[:3]:
    print(post.get_text())
    print(post.prettify())

    print('-- '*10)  # для красоты

Обзор Llemma: новая математическая open-source модель
<h2 class="tm-title tm-title_h2">
 <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/companies/mts_ai/articles/771476/">
  <span>
   Обзор Llemma: новая математическая open-source модель
  </span>
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 
Из фото в 3D, ч.1: геометрия формирования изображения
<h2 class="tm-title tm-title_h2">
 <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/companies/magnus-tech/articles/770930/">
  <span>
   Из фото в 3D, ч.1: геометрия формирования изображения
  </span>
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 
Как провести курс для разработчиков (за 1.5 млн) в маленькой компании и не сойти с ума
<h2 class="tm-title tm-title_h2">
 <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/articles/771466/">
  <span>
   Как провести курс для разработчиков (за 1.5

## Задание на семинар 1

А что если мы хотим зайти еще глубже по дереву тегов и, например, для каждого заголовка поста найти никнейм юзера, который написал этот пост, и время написания поста?

Для этого надо снова зайти в просмотор кода страницы и увидеть, что там просиходит что-то такое:

(Заодно обратите внимание, как пишутся комменты в html)

```
<article class="tm-articles-list__item" data-navigatable="" id="771476" tabindex="0">
 <div class="tm-article-snippet tm-article-snippet">
  <div class="tm-article-snippet__meta-container">
   <div class="tm-article-snippet__meta">
    <span class="tm-user-info tm-article-snippet__author">
     <a class="tm-user-info__userpic" href="/ru/users/darinka666/" title="darinka666">
      <div class="tm-entity-image">
       <img alt="" class="tm-entity-image__pic" height="24" src="https://assets.habr.com/habr-web/img/avatars/093.png" width="24"/>
      </div>
     </a>
     <span class="tm-user-info__user tm-user-info__user_appearance-default">
      <a class="tm-user-info__username" href="/ru/users/darinka666/">
       darinka666
       <!-- -->
      </a>
      <span class="tm-article-datetime-published">
       <time datetime="2023-11-02T09:22:25.000Z" title="2023-11-02, 12:22">
        11 минут назад
       </time>
      </span>
     </span>
    </span>
   </div>
   <!-- -->
  </div>
  <h2 class="tm-title tm-title_h2">
   <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/companies/mts_ai/articles/771476/">
    <span>
     Обзор Llemma: новая математическая open-source модель
    </span>
   </a>
  </h2>
  <div class="tm-article-snippet__stats">
   <div class="tm-article-complexity tm-article-complexity_complexity-medium">
    <span class="tm-svg-icon__wrapper tm-article-complexity__icon">
     <svg class="tm-svg-img tm-svg-icon" height="24" width="24">
      <title>
       Уровень сложности
      </title>
      <use xlink:href="/img/megazord-v28.2fb1b1c1..svg#complexity-medium">
      </use>
     </svg>
    </span>
    <span class="tm-article-complexity__label">
     Средний
    </span>
   </div>
```
(и так далее; часть вывода обрезана: обратите внимание, нет закрывающего тега `</article>`)

In [32]:
for post in soup.find_all("article")[:3]:
    print("TEXT:")
    print(post.get_text()[:1000])
    print('-- '*10)
    print("HTML:")
    print(post.prettify()[:2000])

    print('-- '*10)
    print('-- '*10)
    print('-- '*10)

## Задание на семинар 2

Скачать главную страницу Яндекс.Погоды и
    
- а) распечатать сегодняшнюю температуру и облачность
- б) распечатать время восхода и заката
- в) погоду на завтра

Простая версия: вместо Яндекс.Погоды возьмите [этот](https://simple-weather-website.netlify.app/) сайт и получите температуру, влажность и скорость ветра

## Хорошая статья про это все

[https://sysblok.ru/courses/obkachka-sajtov-svoimi-rukami-razbiraemsja-s-html/](https://sysblok.ru/courses/obkachka-sajtov-svoimi-rukami-razbiraemsja-s-html/)

# Бонус

Иногда можно не изгаляться с html, а воспользоваться специальным инструментом: **API**

API расшифровывается как Application Programming Interface. Это набор правил, которыми одна программа может общаться и взаимодействовать с другой, в том числе с сайтами. Например, "сделай запрос к такой-то странице в таком-то формате, чтобы получить JSON с данными о погоде и не парсить HTML руками"

Но сначала пример попроще:

In [39]:
shibe = requests.get("http://shibe.online/api/shibes?count=1")

Обратите внимание, что `.text` вернет нам *строку*:

In [40]:
shibe.text

'["https://cdn.shibe.online/shibes/e5c085fc427528fee7d4c3935ff4cd79af834a82.jpg"]'

Но у респонза есть еще метод `.json()`. В данном случае он сразу приведет к списку (а если бы API возвращало словарь с несколькими ключами — к словарю)



In [41]:
shibe.json()

['https://cdn.shibe.online/shibes/e5c085fc427528fee7d4c3935ff4cd79af834a82.jpg']

Более сложный, но и более интересный пример: https://newsapi.org/

 На главной нужно получить ключ. Многие API доступны только по ключам: например, API гитхаба так определяет, к каким репозиториям у вас есть доступ, а к каким нет; некоторые API ограничивают число запросов на одного пользователя; и т.д.

 **Ключ — это личная информация**, как, например, пароль. Ключ нежелательно вставлять в код, потому что будет легко забыть его удалить перед тем, как вы где-то этот код опубликуете. Лучше положить в отдельный файл и его читать:

In [61]:
with open("news_api_key.txt", "r") as f:
  api_key = f.read()

Какие параметры можно указывать в запросе, смотрите здесь: https://newsapi.org/docs/endpoints/everything

In [79]:
news = requests.get(
    f"https://newsapi.org/v2/everything?q=Apple&from=2023-11-01&sortBy=popularity&apiKey={api_key}"
    ).json()

In [80]:
news["totalResults"]

574

In [81]:
articles = news["articles"][:5]

In [84]:
articles

[{'source': {'id': 'engadget', 'name': 'Engadget'},
  'author': 'Mariella Moon',
  'title': 'Google and Match Group settle antitrust case before it goes to trial',
  'description': 'The antitrust lawsuit Epic Games and Match Group have filed against Google was supposed to go to trial on November 6, but now it looks like the video game developer might go at it alone. Google and Match, the parent company of Tinder, OkCupid and Hinge, have …',
  'url': 'https://www.engadget.com/google-and-match-group-settle-antitrust-case-before-it-goes-to-trial-041158809.html',
  'urlToImage': 'https://s.yimg.com/ny/api/res/1.2/2.umw5DEZTxqv1czt7En9A--/YXBwaWQ9aGlnaGxhbmRlcjt3PTEyMDA7aD04MDA-/https://s.yimg.com/os/creatr-uploaded-images/2023-10/871dbd80-7864-11ee-9f3f-9642d3093703',
  'publishedAt': '2023-11-01T04:11:58Z',
  'content': 'The antitrust lawsuit Epic Games and Match Group have filed against Google was supposed to go to trial on November 6, but now it looks like the video game developer might

Другие API:

1. OpenWeatherMap (https://openweathermap.org/api) - предоставляет информацию о погоде, прогнозы и исторические данные по всему миру. Есть бесплатный лимит запросов, но все равно требуется ввести данные банковской карты.

2. GitHub (https://developer.github.com/v3/) - API для доступа к данным о репозиториях, пользователях, коммитах и т.д.

3. Open Trivia API (https://opentdb.com/api_config.php) - API для получения вопросов и ответов из базы данных триивиума (викторины).

In [88]:
requests.get("https://opentdb.com/api.php?amount=10").json()

{'response_code': 0,
 'results': [{'category': 'Entertainment: Video Games',
   'type': 'multiple',
   'difficulty': 'hard',
   'question': 'What was the name of the hero in the 80s animated video game &#039;Dragon&#039;s Lair&#039;?',
   'correct_answer': 'Dirk the Daring',
   'incorrect_answers': ['Arthur', 'Sir Toby Belch', 'Guy of Gisbourne']},
  {'category': 'Entertainment: Music',
   'type': 'boolean',
   'difficulty': 'easy',
   'question': 'Eurobeat is primarily an Italian genre of music.',
   'correct_answer': 'True',
   'incorrect_answers': ['False']},
  {'category': 'Celebrities',
   'type': 'multiple',
   'difficulty': 'medium',
   'question': 'Donald J. Trump&#039;s Middle Name is...',
   'correct_answer': 'John',
   'incorrect_answers': ['Jeff', 'Jerald', 'Jason']},
  {'category': 'Vehicles',
   'type': 'multiple',
   'difficulty': 'hard',
   'question': 'What engine is in the Lexus SC400?',
   'correct_answer': '1UZ-FE',
   'incorrect_answers': ['2JZ-GTE', '7M-GTE', '5M-

А еще есть Drama Corpus, который делали в том числе при участии Школы Лингвистики: https://dracor.org/doc/api

In [91]:
requests.get("https://dracor.org/api/corpora/rus/play/gogol-revizor/cast").json()[:5]

[{'id': 'gorodnichij',
  'name': 'Городничий',
  'isGroup': False,
  'gender': 'MALE',
  'numOfScenes': 21,
  'numOfSpeechActs': 172,
  'numOfWords': 4991,
  'degree': 26,
  'weightedDegree': 84,
  'closeness': 0.8760416666666667,
  'betweenness': 0.1264602852635307,
  'eigenvector': 0.24182361124623364},
 {'id': 'ammos_fedorovich_ljapkin_tjapkin',
  'name': 'Аммос Федорович',
  'isGroup': False,
  'gender': 'MALE',
  'numOfScenes': 8,
  'numOfSpeechActs': 49,
  'numOfWords': 748,
  'degree': 21,
  'weightedDegree': 54,
  'closeness': 0.7377192982456141,
  'betweenness': 0.010129431082777935,
  'eigenvector': 0.23046261804194926},
 {'id': 'artemij_filippovich_zemljanika',
  'name': 'Артемий Филиппович Земляника',
  'isGroup': False,
  'gender': 'MALE',
  'numOfScenes': 10,
  'numOfSpeechActs': 51,
  'numOfWords': 737,
  'degree': 21,
  'weightedDegree': 63,
  'closeness': 0.7377192982456141,
  'betweenness': 0.010129431082777935,
  'eigenvector': 0.23046261804194926},
 {'id': 'luka_luk